In [1]:
import pandas as pd
import numpy as np
import pickle
# from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
path = "/om/user/shobhita/src/chexpert/data/CheXpert-v1.0-small/"

In [2]:
# path='/local/nhulkund/UROP/Chexpert/data/CheXpert-v1.0-small/'
train=pd.read_csv(path+'train.csv')
label_cols=train.columns[5:]
train[label_cols]=train[label_cols].fillna(0.0)
train[label_cols]=train[label_cols].replace(-1.0,1.0)
train["ID"] = train["Path"].apply(lambda x: x.split("patient")[1].split("/")[0])

In [41]:
train_patients, test_patients = train_test_split(train_shuffled["ID"].unique(), test_size=0.1)
train_split, test_split = train_shuffled[train_shuffled["ID"].isin(train_patients)], train_shuffled[train_shuffled["ID"].isin(test_patients)]

In [44]:
train_split.to_csv(path + "train_preprocessed.csv")
test_split.to_csv(path + "test_preprocessed.csv")

In [6]:
names = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices']


def get_class_split(labels, proportion=True):
    split = {}
    total = len(labels)
    for name in names:
        split[name] = sum(labels[name])/total if proportion else sum(labels[name])
    return split

# Add GAN data to dataset

In [3]:
train = pd.read_csv(path + "train_preprocessed.csv")
# test = pd.read_csv(path + "test_train_preprocessed.csv")

In [7]:
base_path = "CheXpert-v1.0-small/train/patient65000/study1/"
label_path = "/om/user/shobhita/src/chexpert/data/"
with open(path + "gan_labels_65000_prop.pkl", "rb") as handle:
    gan_labels = pickle.load(handle)

gan_data = []
    
for img_id, label in gan_labels.items():
    img_filename = "fake_{}.png".format(img_id)
    img_path = base_path + img_filename
    img_row = {}
    img_row["Path"] = img_path
    img_row["Sex"] = "Female"
    img_row["Age"] = 21
    img_row["Frontal/Lateral"] = "Frontal"
    img_row["AP/PA"] = "AP"
    for i, name in enumerate(names):
        img_row[name] = label[i]
    img_row["ID"] = 65000
    gan_data.append(img_row)

FileNotFoundError: [Errno 2] No such file or directory: '/om/user/shobhita/src/chexpert/data/CheXpert-v1.0-small/gan_labels_65000_prop.pkl'

In [49]:
gan_df = pd.DataFrame(gan_data)
full_train_df = pd.concat([train, gan_df], ignore_index=True)
full_train_df.to_csv(path + "train_with_gan_preprocessed.csv", index=False)

In [50]:
train_gan = pd.read_csv(path + "train_with_gan_preprocessed.csv")
train_gan.shape

(254510, 20)

In [9]:
names = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices']
print(label)


[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# Split dataset into train and test

In [40]:
train["ID"][0]

40439

In [10]:
train_ids = train["Path"].apply(lambda x: int(x.split("patient")[1].split("/")[0]))
test_ids = test["Path"].apply(lambda x: int(x.split("patient")[1].split("/")[0]))

In [13]:
max(train_ids.max(), test_ids.max())

64540

In [31]:
valid=pd.read_csv(path+'valid.csv')
label_cols=train.columns[5:]
valid[label_cols]=valid[label_cols].fillna(0.0)
valid[label_cols]=valid[label_cols].replace(-1.0,1.0)
valid.to_csv(path+"valid_preprocessed.csv")

# Downsample train dataset

### Downsample to 50%

In [3]:
path = "/om/user/shobhita/src/chexpert/data/CheXpert-v1.0-small/"
train = pd.read_csv(path + "train_preprocessed.csv")
test = pd.read_csv(path + "test_train_preprocessed.csv")
train_gan = pd.read_csv(path + "train_with_gan_preprocessed.csv")

In [7]:
train_subset_patients, _ = train_test_split(train["ID"].unique(), test_size=0.5)
train_subset = train[train["ID"].isin(train_subset_patients)]

In [8]:
len(train_subset)/len(train)

0.5007671643717941

In [10]:
gan_data = train_gan[train_gan["ID"] == 64600]
train_subset_with_gan = pd.concat([train_subset, gan_data], ignore_index=True)

In [12]:
train_subset.to_csv(path + "train_preprocessed_subset.csv", index=False)
train_subset_with_gan.to_csv(path + "train_preprocessed_subset_with_gan.csv", index=False)

### Downsample to 10%

In [5]:
path = "/om/user/shobhita/src/chexpert/data/CheXpert-v1.0-small/"
train = pd.read_csv(path + "train_preprocessed.csv")
test = pd.read_csv(path + "test_train_preprocessed.csv")
train_gan = pd.read_csv(path + "train_with_gan_preprocessed.csv")

train_subset_patients, _ = train_test_split(train["ID"].unique(), test_size=0.9)
train_subset = train[train["ID"].isin(train_subset_patients)]

print("Ratio: ", len(train_subset)/len(train))

Ratio:  0.1011514913774697


In [14]:
splits = get_class_split(train_subset)
splits

{'No Finding': 2020.0,
 'Enlarged Cardiomediastinum': 2018.0,
 'Cardiomegaly': 3148.0,
 'Lung Opacity': 10246.0,
 'Lung Lesion': 927.0,
 'Edema': 5898.0,
 'Consolidation': 3818.0,
 'Pneumonia': 2261.0,
 'Atelectasis': 5931.0,
 'Pneumothorax': 2260.0,
 'Pleural Effusion': 8869.0,
 'Pleural Other': 603.0,
 'Fracture': 887.0,
 'Support Devices': 10625.0}

In [16]:
train_subset.to_csv(path + "train_preprocessed_subset_10.csv", index=False)

In [16]:
train = pd.read_csv(path + "train_preprocessed_subset_50.csv")

In [17]:
len(train)

100850

In [18]:
patient_id = 64900
gan_label_path = "/om/user/shobhita/src/chexpert/data/gan_labels_{}_prop.pkl".format(patient_id)
base_path = "CheXpert-v1.0-small/train/patient{}/study1/".format(patient_id)

with open(gan_label_path, "rb") as handle:
    gan_labels = pickle.load(handle)

gan_data = []
    
for img_id, label in gan_labels.items():
    img_filename = "{}.jpg".format(img_id)
    img_path = base_path + img_filename
    img_row = {}
    img_row["Path"] = img_path
    img_row["Sex"] = "Female"
    img_row["Age"] = 21
    img_row["Frontal/Lateral"] = "Frontal"
    img_row["AP/PA"] = "AP"
    for i, name in enumerate(names):
        img_row[name] = label[i]
    img_row["ID"] = patient_id
    gan_data.append(img_row)

In [19]:
len(gan_data)

47000

In [20]:
gan_df = pd.DataFrame(gan_data)
full_train_df = pd.concat([train, gan_df], ignore_index=True)
full_train_df.to_csv(path + "train_preprocessed_subset_50_with_gan.csv", index=False)

In [21]:
train_gan = pd.read_csv(path + "train_preprocessed_subset_50_with_gan.csv")

In [22]:
len(train)

100850

In [23]:
len(train_gan)

147850

In [25]:
train_gan[train_gan["ID"] == 64900]["Path"].iloc[0]

'CheXpert-v1.0-small/train/patient64900/study1/Lung_Lesion_0.jpg'

### Downsample to 5%

In [19]:
path = "/om/user/shobhita/src/chexpert/data/CheXpert-v1.0-small/"
train = pd.read_csv(path + "train_preprocessed.csv")
test = pd.read_csv(path + "test_train_preprocessed.csv")
train_gan = pd.read_csv(path + "train_with_gan_preprocessed.csv")

train_subset_patients, _ = train_test_split(train["ID"].unique(), test_size=0.95)
train_subset = train[train["ID"].isin(train_subset_patients)]

print("Ratio: ", len(train_subset)/len(train))

splits = get_class_split(train_subset, proportion=False)
print(splits)

Ratio:  0.04991782155111202
{'No Finding': 1036.0, 'Enlarged Cardiomediastinum': 1010.0, 'Cardiomegaly': 1485.0, 'Lung Opacity': 5117.0, 'Lung Lesion': 456.0, 'Edema': 2877.0, 'Consolidation': 2021.0, 'Pneumonia': 1096.0, 'Atelectasis': 3064.0, 'Pneumothorax': 1098.0, 'Pleural Effusion': 4381.0, 'Pleural Other': 324.0, 'Fracture': 338.0, 'Support Devices': 5379.0}


In [20]:
train_subset.to_csv(path + "train_preprocessed_subset_5.csv", index=False)


In [22]:
train = pd.read_csv(path + "train_preprocessed_subset_5.csv")
len(train)

10053

In [23]:
patient_id = 65100
gan_label_path = "/om/user/shobhita/src/chexpert/data/gan_labels_5.pkl"
base_path = "CheXpert-v1.0-small/train/patient{}/study1/".format(patient_id)

with open(gan_label_path, "rb") as handle:
    gan_labels = pickle.load(handle)

gan_data = []
    
for img_id, label in gan_labels.items():
    img_filename = "{}.jpg".format(img_id)
    img_path = base_path + img_filename
    img_row = {}
    img_row["Path"] = img_path
    img_row["Sex"] = "Female"
    img_row["Age"] = 21
    img_row["Frontal/Lateral"] = "Frontal"
    img_row["AP/PA"] = "AP"
    for i, name in enumerate(names):
        img_row[name] = label[i]
    img_row["ID"] = patient_id
    gan_data.append(img_row)

In [24]:
gan_df = pd.DataFrame(gan_data)
full_train_df = pd.concat([train, gan_df], ignore_index=True)
full_train_df.to_csv(path + "train_preprocessed_subset_5_with_gan.csv", index=False)

In [25]:
len(full_train_df)

14903

In [26]:
len(gan_df)

4850

### Downsample to 1%

In [28]:
path = "/om/user/shobhita/src/chexpert/data/CheXpert-v1.0-small/"
train = pd.read_csv(path + "train_preprocessed.csv")
train_gan = pd.read_csv(path + "train_with_gan_preprocessed.csv")

train_subset_patients, _ = train_test_split(train["ID"].unique(), test_size=0.99)
train_subset = train[train["ID"].isin(train_subset_patients)]

print("Size: ", len(train_subset))
print("Ratio: ", len(train_subset)/len(train))

splits = get_class_split(train_subset, proportion=False)
print(splits)

Size:  2050
Ratio:  0.010179203638692891
{'No Finding': 228.0, 'Enlarged Cardiomediastinum': 250.0, 'Cardiomegaly': 311.0, 'Lung Opacity': 1044.0, 'Lung Lesion': 134.0, 'Edema': 496.0, 'Consolidation': 343.0, 'Pneumonia': 244.0, 'Atelectasis': 571.0, 'Pneumothorax': 272.0, 'Pleural Effusion': 926.0, 'Pleural Other': 53.0, 'Fracture': 82.0, 'Support Devices': 1075.0}


In [29]:
train_subset.to_csv(path + "train_preprocessed_subset_1.csv", index=False)

In [31]:
train = pd.read_csv(path + "train_preprocessed_subset_1.csv")
len(train)

2050

In [32]:
patient_id = 65200
gan_label_path = "/om/user/shobhita/src/chexpert/data/gan_labels_1.pkl"
base_path = "CheXpert-v1.0-small/train/patient{}/study1/".format(patient_id)

with open(gan_label_path, "rb") as handle:
    gan_labels = pickle.load(handle)

gan_data = []
    
for img_id, label in gan_labels.items():
    img_filename = "{}.jpg".format(img_id)
    img_path = base_path + img_filename
    img_row = {}
    img_row["Path"] = img_path
    img_row["Sex"] = "Female"
    img_row["Age"] = 21
    img_row["Frontal/Lateral"] = "Frontal"
    img_row["AP/PA"] = "AP"
    for i, name in enumerate(names):
        img_row[name] = label[i]
    img_row["ID"] = patient_id
    gan_data.append(img_row)

In [33]:
gan_df = pd.DataFrame(gan_data)
full_train_df = pd.concat([train, gan_df], ignore_index=True)
full_train_df.to_csv(path + "train_preprocessed_subset_1_with_gan.csv", index=False)

In [34]:
len(full_train_df)

3600

In [ ]:
"""
1) For each label vector choose one class, bring them to uniform distribution
2) Pick a category and target number
"""